# Homework 4

## Description

### Data
[Reuters-21578 Text Categorization Collection Data Set](https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection) - about 2 million instances, 28MB in size


News: 21 SGML files
We only deal with news contents inside <body> </body> tags
The other files will not be needed in this homework

### Format
One text file consisting of lines of records.

Each record contains 9 attributes separated by semicolons: 


Reuters-21578, Distribution 1.0 includes five files (all-exchanges-strings.lc.txt, all-orgs-strings.lc.txt, all-people-strings.lc.txt, all-places-strings.lc.txt, and all-topics-strings.lc.txt) which list the names of *all* legal categories in each set. A sixth file, cat-descriptions_120396.txt gives some additional information on the category sets.


### Task
4 subtasks:
+ (30pt) Given the Reuters-21578 dataset, please calculate the term frequencies, and output the representation of the document contents as a term-document count matrix.
+ (30pt) Implement matrix multiplication by MapReduce. Your program should be able to output the result in appropriate dimensions.
+ (40pt) Given the term-document matrix in (1), compute the SVD decomposition of the matrix using MapReduce. Output the resulting eigenvalues and eigenvectors.
+ (Bonus) Given the term-document matrix in (1), compute the CUR decomposition of the matrix using MapReduce. Output the resulting eigenvalues and eigenvectors.

## Output Format

1. Term-Document Matrix:
A MxN matrix: with rows as term frequencies and columns as documents (N=21,578) 
2. Result of Matrix Multiplication:
A MxR matrix: (MxN) * (NxR)
3. Eigen Pairs:
Eigenvalues sorted in descending order, and their corresponding eigenvectors
4. Eigen Pairs: 
Eigenvalues sorted in descending order, and their corresponding eigenvectors

## Implementation

In [41]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.hadoop.fs._
import org.apache.hadoop.conf.Configuration

import java.io.{File,PrintWriter}

  def printSample(writer: Any, data: Any, title: String = "", format: String = ""){
    println("\n"+title+" Data Sample: " + format)
    println(data+"\n")
  }

  def printSpark(writer: Any, spark: SparkSession): Unit = {
    println("Spark Entity:       " + spark)
    println("Spark version:      " + spark.version)
    println("Spark master:       " + spark.sparkContext.master)
    println("Running 'locally'?: " + spark.sparkContext.isLocal)
    println("")
  }

  def outputWriter(fileString: String): PrintWriter ={
    val outputPath = new Path(fileString)
    val outputStream = outputPath.getFileSystem(new Configuration()).create(outputPath);
    new PrintWriter(outputStream)
  }

  def getFile(fileString: String): Array[String] ={
    val inputPath = new Path(fileString)
    val inputBuffer = scala.collection.mutable.ArrayBuffer.empty[String]
    val iterator = inputPath.getFileSystem(new Configuration()).listFiles(inputPath, false)
    while(iterator.hasNext()){
        val fileStatus = iterator.next()
        if(fileStatus.isFile()){
          inputBuffer += fileStatus.getPath().toString()
        }
    }
    inputBuffer.toArray
  }


val writer = null
printSpark(writer, spark)

Spark Entity:       org.apache.spark.sql.SparkSession@2703d2d0
Spark version:      2.2.0
Spark master:       local[*]
Running 'locally'?: true



printSample: (writer: Any, data: Any, title: String, format: String)Unit
printSpark: (writer: Any, spark: org.apache.spark.sql.SparkSession)Unit
outputWriter: (fileString: String)java.io.PrintWriter
getFile: (fileString: String)Array[String]
writer: Null = null


In [42]:
val spark2 = spark

var data = getFile("./data")
var sample_data = Array("file:/home/micky/big_data/hw3/data/reut2-017.sgm")

spark2 = org.apache.spark.sql.SparkSession@2703d2d0
data = Array(file:/home/micky/big_data/hw4/data/reut2-017.sgm, file:/home/micky/big_data/hw4/data/reut2-001.sgm, file:/home/micky/big_data/hw4/data/reut2-021.sgm, file:/home/micky/big_data/hw4/data/reut2-003.sgm, file:/home/micky/big_data/hw4/data/reut2-010.sgm, file:/home/micky/big_data/hw4/data/reut2-013.sgm, file:/home/micky/big_data/hw4/data/reut2-004.sgm, file:/home/micky/big_data/hw4/data/reut2-020.sgm, file:/home/micky/big_data/hw4/data/reut2-008.sgm, file:/home/micky/big_data/hw4/data/reut2-014.sgm, file:/home/micky/big_data/hw4/data/reut2-007.sgm, file:/home/micky/big_data/hw4/data/reut2-018.sgm, file:/home/micky/big_data/hw4/data/reut2-005.sgm, file:/home/micky/big_data/hw4/da...


[file:/home/micky/big_data/hw4/data/reut2-017.sgm, file:/home/micky/big_data/hw4/data/reut2-001.sgm, file:/home/micky/big_data/hw4/data/reut2-021.sgm, file:/home/micky/big_data/hw4/data/reut2-003.sgm, file:/home/micky/big_data/hw4/data/reut2-010.sgm, file:/home/micky/big_data/hw4/data/reut2-013.sgm, file:/home/micky/big_data/hw4/data/reut2-004.sgm, file:/home/micky/big_data/hw4/data/reut2-020.sgm, file:/home/micky/big_data/hw4/data/reut2-008.sgm, file:/home/micky/big_data/hw4/data/reut2-014.sgm, file:/home/micky/big_data/hw4/data/reut2-007.sgm, file:/home/micky/big_data/hw4/data/reut2-018.sgm, file:/home/micky/big_data/hw4/data/reut2-005.sgm, file:/home/micky/big_data/hw4/data/reut2-015.sgm, file:/home/micky/big_data/hw4/data/reut2-011.sgm, file:/home/micky/big_data/hw4/data/reut2-002.sgm, file:/home/micky/big_data/hw4/data/reut2-012.sgm, file:/home/micky/big_data/hw4/data/reut2-019.sgm, file:/home/micky/big_data/hw4/data/reut2-009.sgm, file:/home/micky/big_data/hw4/data/reut2-016.sgm,

In [43]:
import scala.collection.mutable.ListBuffer

//TODO: Sample -> Actual
var regex = """(?s)<REUTERS.*?NEWID="(\d*)".*?>.*?<TEXT.*?>(.*?)<\/TEXT>.*?<\/REUTERS>""".r
var body_regex = """(?s).*?<BODY.*?>(.*?)<\/BODY>.*?""".r
var newsParse = spark.sparkContext.emptyRDD[(Int, String)]

sample_data.map{
    path  =>
    val singleNewsParse = spark.sparkContext.wholeTextFiles(path).flatMap{
        case (_, string) =>
        val list =  ListBuffer[Tuple2[Int, String]]()
        for(m <- regex.findAllIn(string).matchData){
            for( n <- body_regex.findAllIn(m.group(2)).matchData){
                list += (Tuple2(m.group(1).toInt, n.group(1).replace("\n Reuter\n&#3;","")))
            }
        }
        list.toSeq
    }
    newsParse = newsParse.union(singleNewsParse)
}
var newsCount = newsParse.count()
print("\nCount: "+ newsCount+"\n")
printSample(writer, newsParse.take(2).mkString("\n\n"), "Parsed Sample")


Count: 940

Parsed Sample Data Sample: 
(17001,The Brazilian Coffee Institute, IBC,
plans to sell in a series of auctions over the next few weeks
robusta coffee purchased in London last year, but details of
where and when auctions will take place are still to be
finalised, IBC president Jorio Dauster told reporters.
    The sales of 630,000 bags of robusta and an unspecified
amount of Brazilian arabica coffee will take place over a
minimum of six months but it is not decided where sales will
take place or whether they will be held weekly or monthly.
    The amount offered at each sale has also not been set, but
could be in the order of 100,000 bags, Dauster said.)

(17002,AMCA International Ltd said it
appointed president and chief executive officer WIlliam Holland
to succeed Kenneth Barclay as chairman.
    Barclay, who is 60 years old, decided not to stand for
reappointment as chairman this year but will continue as a
director, AMCA said.)



regex = (?s)<REUTERS.*?NEWID="(\d*)".*?>.*?<TEXT.*?>(.*?)<\/TEXT>.*?<\/REUTERS>
body_regex = (?s).*?<BODY.*?>(.*?)<\/BODY>.*?
newsParse = UnionRDD[23] at union at <console>:78
newsCount = 940


940

### Task 1 - Term-Frequency Matrix

In [44]:
def wordPreProcess(input: Any): Array[String] = {
        var matchRegex = """([$]?(?:[\w]+(?:[\w',]*[\w]+)+|[\w]))""".r
        var list =  ListBuffer[String]()
        for(m <- matchRegex.findAllIn(input.toString.toLowerCase).matchData;
          e <- m.subgroups)
          list+=e
        list.toArray
    }

var newsCount = spark.sparkContext.broadcast(newsParse.count())
var matrixIndex = spark.sparkContext.broadcast(newsParse.map{
    case(id, string) =>
    id.toInt
}.sortBy{
    case (id) => id
}.zipWithIndex.collectAsMap())

printSample(writer, matrixIndex.value.take(5).mkString("\n"), "Ziped With Index Sample")


Ziped With Index Sample Data Sample: 
17758 -> 719
17417 -> 411
17408 -> 402
17004 -> 3
17749 -> 711



newsCount = Broadcast(22)
matrixIndex = Broadcast(25)


wordPreProcess: (input: Any)Array[String]


Broadcast(25)

In [45]:
var flattenTerms = newsParse.flatMap{
    case (id, string)  =>
    wordPreProcess(string).map{
        word =>
        ((word, matrixIndex.value(id).toInt), 1)
    }  
}.reduceByKey{
    (x, y) => x + y
}.map{
    case((word, id),count) =>
    (word,(id, count))
}.groupByKey()
printSample(writer, flattenTerms.take(20).mkString("\n"), "Parsed Shingle Sample")


Parsed Shingle Sample Data Sample: 
(secure,CompactBuffer((896,2), (106,1), (508,1), (413,1)))
(gaping,CompactBuffer((820,1)))
(dayton,CompactBuffer((607,1)))
(fred,CompactBuffer((353,1)))
(end,CompactBuffer((322,4), (330,1), (729,1), (96,1), (811,1), (178,1), (193,1), (860,1), (266,1), (422,1), (79,2), (176,1), (117,1), (795,1), (39,1), (600,1), (410,2), (90,1), (402,1), (239,1), (302,1), (909,1), (569,1), (813,1), (198,1), (908,1), (485,3), (357,1), (253,1), (454,1), (776,1), (257,1), (210,3), (207,1), (41,1), (868,1), (235,1), (92,1), (351,1), (423,1), (879,1), (26,1), (842,1), (625,1), (185,1), (171,2), (160,1), (352,2), (367,1), (380,1), (157,1), (877,1), (832,2), (280,1), (240,2), (396,2), (353,1), (42,1), (194,2), (57,1), (404,2), (437,1), (136,1), (201,1), (481,1), (432,1), (459,2), (348,1), (677,1), (18,1), (35,1), (243,1), (119,1), (632,1), (848,2), (467,1), (70,1), (681,1), (318,1), (271,3), (413,1), (875,5), (268,1), (466,1), (463,2), (452,1), (342,1), (517,1)))
(been,Comp

flattenTerms = ShuffledRDD[32] at groupByKey at <console>:75


ShuffledRDD[32] at groupByKey at <console>:75

In [46]:
var termDocumentMatrix = flattenTerms.map{
    case (word, iterable) =>
    var row = Array.fill(newsCount.value.toInt){0}
    iterable.toVector.map{
        case(id, count) =>
        row(id) = count
    }
    row
}

termDocumentMatrix = MapPartitionsRDD[33] at map at <console>:53


MapPartitionsRDD[33] at map at <console>:53

In [ ]:
val termDocumentString = termDocumentMatrix.map{
      array =>
      array.mkString(",")
}

termDocumentString.saveAsTextFile("./output/termDocumentString")

### Task 2 - Matrix Multiplication

In [47]:
val rand = scala.util.Random
val r_value = 3
var otherMatrix = spark.sparkContext.broadcast(Array.fill(newsCount.value.toInt,r_value){rand.nextInt(2000) - 1000})

rand = scala.util.Random$@79b3490b
r_value = 3
otherMatrix = Broadcast(29)


Broadcast(29)

In [48]:
var multipliedMatrix = termDocumentMatrix.map{
    array =>
    var newRow = Array.fill(otherMatrix.value(0).length){0}
    newRow.zipWithIndex.map{
        case(value, columnid) =>
        var newValue = value
        array.zipWithIndex.foreach{
            case(arrayValue, rowid) =>
            newValue += arrayValue * otherMatrix.value(rowid)(columnid)
        }
        newValue
    }
}

multipliedMatrix.take(2).foreach{
    array =>
    println(array.mkString(", "))
}

-1931, -2051, 1485
-571, 946, -341


multipliedMatrix = MapPartitionsRDD[34] at map at <console>:75


MapPartitionsRDD[34] at map at <console>:75

103820004 Michael Fu